In [137]:
import os
import re
import random
import pandas as pd
import numpy as np
import json
import unicodedata
from tqdm.auto import tqdm
from ast import literal_eval

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import chisquare

import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

from multiprocessing import Pool, cpu_count

### wikipedia-korean
- title: 문서 제목
- section_titles: 섹션 제목들의 리스트
- section_texts: 각 섹션의 본문 텍스트 리스트
- text: 문서 전체 텍스트

In [3]:
ds = load_dataset("lcw99/wikipedia-korean-20240501")

README.md:   0%|          | 0.00/475 [00:00<?, ?B/s]

data/train-00000-of-00007.parquet:   0%|          | 0.00/468M [00:00<?, ?B/s]

data/train-00001-of-00007.parquet:   0%|          | 0.00/289M [00:00<?, ?B/s]

data/train-00002-of-00007.parquet:   0%|          | 0.00/232M [00:00<?, ?B/s]

data/train-00003-of-00007.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

data/train-00004-of-00007.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

data/train-00005-of-00007.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

data/train-00006-of-00007.parquet:   0%|          | 0.00/198M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/515425 [00:00<?, ? examples/s]

In [8]:
df = pd.DataFrame(ds['train'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515425 entries, 0 to 515424
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   title           515425 non-null  object
 1   section_titles  515425 non-null  object
 2   section_texts   515425 non-null  object
 3   text            515425 non-null  object
dtypes: object(4)
memory usage: 15.7+ MB


In [10]:
df.head()

,title,section_titles,section_texts,text
0,지미 카터,"[Introduction, 약력, 생애, 대통령 재임, 퇴임 이후, 평가, 같이 보...","[\n\n'''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는...","'''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ..."
1,수학,"[Introduction, 역사, 세부 분야, 영향, 같이 보기, 참고 문헌, 외부...","[\n\n'''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변...","'''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변화 등의 ..."
2,수학 상수,"[Introduction, 수학 상수표, 관련 상수들, 기타 상수들, 같이 보기]","[\n'''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의...","'''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의 반대..."
3,문학,"[Introduction, 일반적인 문학의 분류, 대중문학의 분류, 문학 사조, 문...","[\n\n\n파일:Fragonard, The Reader.jpg|섬네일|250px|...","파일:Fragonard, The Reader.jpg|섬네일|250px|장오노레 프라..."
4,나라 목록,"[Introduction, 기준, 남극, EU, 참고, 몰타 기사단, [[마이크로네...",[\n스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n\n이...,스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n이 목록에 ...


In [13]:
df['text'].apply(lambda x: len(x)).describe()

count    515425.000000
mean       1448.979076
std        3128.588490
min         103.000000
25%         370.000000
50%         646.000000
75%        1362.000000
max      566852.000000
Name: text, dtype: float64

In [16]:
wiki = load_dataset("lcw99/wikipedia-korean-20240501", split="train")

wiki_50k = wiki.shuffle(seed=42).select(range(50_000))

In [30]:
sample_df = wiki_50k.to_pandas()

In [31]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           50000 non-null  object
 1   section_titles  50000 non-null  object
 2   section_texts   50000 non-null  object
 3   text            50000 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [43]:
df_long = sample_df.explode(["section_titles", "section_texts"], ignore_index=True)
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216720 entries, 0 to 216719
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   title           216720 non-null  object
 1   section_titles  216720 non-null  object
 2   section_texts   216720 non-null  object
 3   text            216720 non-null  object
 4   text_norm       216720 non-null  object
 5   recon_norm      216720 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


In [44]:
df_long = df_long.rename(columns={
    "section_titles": "section_title",
    "section_texts": "section_text",
})


In [45]:
df_long.head()

,title,section_title,section_text,text,text_norm,recon_norm
0,로스앤젤레스 아줄 레전즈,Introduction,\n\n\n로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는...,로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는 순수 아마...,로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는 순수 아마...,로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는 순수 아마...
1,이케다 쇼헤이,Introduction,\n\n\n'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...
2,이케다 쇼헤이,외부 링크,* \n\n\n\n\n분류:1981년 출생\n분류:살아있는 사람\n분류:일본의 남자...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...,'''이케다 쇼헤이''' (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다...
3,심고,Introduction,\n'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다...,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...
4,심고,각주,\n\n\n\n분류:불교 용어,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...,'''심고'''(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. ...


In [47]:
df_long['title'].value_counts()[:20]

title
슈퍼주니어의 음반 목록         93
베이블레이드 버스트의 베이 목록    58
2022년 K리그2의 경기 결과    47
유니코드 11000~11FFF     47
2016년 K리그2의 경기 결과    46
불교 용어 목록 (육)         41
미국의 가톨릭 교구 목록        40
수사법                  37
2009년 K리그의 경기 결과     33
로드의 수상 및 후보 목록       32
불교 용어 목록 (선)         31
프랑스 사람 목록            29
2007년 12월            28
2012년 2월             28
2007년 11월            28
JAY B의 음반 목록         27
시에라 엔터테인먼트           26
안녕, 절망선생의 보너스 목록     26
2006년 10월            24
6.25 전쟁 유엔군 파병 국가    23
Name: count, dtype: int64

In [56]:
df_long['section_text'].apply(lambda x: len(x)).describe()

count    216720.000000
mean        351.246189
std        1055.865494
min           0.000000
25%          48.000000
50%         124.000000
75%         320.000000
max      112885.000000
Name: section_text, dtype: float64

In [63]:
df_long.sample(n=20, random_state=42)['section_text'].tolist()

['군마현의 거의 중앙에 위치하고 하루나산의 남동의 산록과 도네강 지역에 정역이 있다. 서반부는 하루나산의 저변의 일부로 높이 200~900m의 경사지이다.한편 동반부는 높이 100~200m의 홍적층 대지이다.\n\n=== 인접하는 자치체 ===\n* 마에바시시\n* 시부카와시\n* 기타군마군: 신토촌\n',
 '=== 12.12 군사반란 ===\n1979년 자 10·26 사건으로 인해 박정희 대통령이 사망한 뒤, 같은 해 전두환 등 하나회를 중심으로 한 신군부는 12·12 군사 반란을 일으켜 군부를 장악하였고 실권자로 떠올랐다. 1980년 초부터 보안사령관 전두환은 K-공작 계획을 실행하여 언론을 조종·통제하기 시작했다. 전두환은 같은 해 4월 14일에 중앙정보부장 서리에 임명돼 대한민국 내의 정보 기관을 모두 장악했다.\n\n=== 민주화시위 ===\n1980년 5월부터 정치 관여 의도를 드러내는 신군부의 움직임에 대한 반발로 전두환 퇴진을 요구하는 학생 시위가 발생했다. 같은 달 국회에서는 계엄 해제와 개헌 논의를 비롯한 정치 현안에 대한 논의를 본격적으로 진행하기 시작했다. 하지만 신군부는 정국 운영에 방해가 되는 세력들을 제거하기 위해 집권 시나리오에 따라 5월 17일 24시에 비상계엄을 전국으로 확대하였고, 계엄 포고령 10호를 선포하여 정치활동 금지령·휴교령·언론 보도검열 강화 같은 조치를 내렸다. 신군부는 김대중, 김영삼, 김종필 등을 포함한 정치인과 재야 인사들 수천 명을 감금하고 군 병력으로 국회를 봉쇄했다. 광주 지역 대학생들은 5월 18일에 \'김대중 석방\', \'전두환 퇴진\', \'비상계엄 해제\' 등의 구호를 외치며 시위를 일으켰다. 신군부는 부마민주항쟁 때처럼 광주의 민주화 요구 시위도 강경 진압하면 잠잠해질 것으로 판단하였고, 공수부대 같은 계엄군을 동원해 진압했다. 신군부는 1980년 3월부터 5월 18일 직전까지 공수부대에 충정훈련을 실시했고, 5월 초부터 군을 사전 이동 배치하고 신군부에 반발하는 시위를 진압할 준비를 마친

### Data Preprocessing

In [ ]:
def clean_wiki_text(text):
    if not isinstance(text, str): return ""
    
    text = unicodedata.normalize("NFKC", text)

    # HTML/XML 태그 제거 (특히 <ref> 태그 안의 내용은 노이즈이므로 내용까지 삭제)
    text = re.sub(r'<ref.*?>.*?</ref>', '', text, flags=re.DOTALL) # <ref>내용</ref> 전체 삭제
    text = re.sub(r'<[^>]+>', '', text) # 남은 <html> 태그들 삭제

    # 링크 괄호 제거
    text = text.replace("[[", "").replace("]]", "")

    # 위키 문법 제거 (=== 소제목 ===, ''', [[링크]])
    text = re.sub(r'={2,}', '', text)  # === 제거
    text = text.replace("'''", "")     # ''' 제거
    
    # 불필요한 공백 및 개행 정리
    # 연속된 공백/탭을 스페이스 하나로
    text = re.sub(r'[ \t]+', ' ', text)
    # 연속된 줄바꿈을 줄바꿈 하나로 (\n\n\n -> \n)
    text = re.sub(r'\n+', '\n', text)
    
    text = text.strip()
    
    return text


In [76]:
# 적용
df_long['section_text_clean'] = df_long['section_text'].apply(clean_wiki_text)

print(df_long.sample(5)['section_text_clean'].tolist())

['유럽과 아시아의 러시아\n유럽 러시아()는 유럽에 속해 있는 러시아의 부분을 이야기한다. 전통적으로 유럽의 경계선은 우랄산맥이지만, 이 정의는 현재 논쟁 중이다.\n러시아의 거의 대부분의 영토는 아시아에 속해 있지만 인구의 거의 대부분은 유럽에 집중되어 있다. 또한 러시아의 대도시들은 유럽에 편중되어 있는데 그 예로 모스크바와 상트페테르부르크가 있다. 이 두 도시는 모두 과거에 러시아의 수도였거나 현재까지 러시아의 수도인 대도시이다.\n러시아 제국 시대에는 러시아의 통제를 받는 동슬라브족 영토를 이야기하는 말로 쓰였다. 이는 벨라루스(백러시아)와 우크라이나의 거의 대부분 지역(드니프르 우크라이나)을 포함한다.', '* 세례시 성호를 긋는 행위를 금할 것.\n* 견진성사를 금할 것.\n* 평신도에 의한 세례를 금할 것.\n* 결혼식에 반지 사용을 금할 것.\n* 예수의 이름으로 고개 숙이는 행위를 금할 것.\n* 중백의와 모자 사용을 금할 것.\n* 다중직을 제공하는 것과 급료 제공을 금할 것.', '순번\n 재임 기간\n 이름\n 1대\n 1997.2.1-1999.4.30\n 최명룡\n 2대\n 1999.5.1-2001.1.3\n 최종규\n 3대\n 2001.1.4-2001.12.26\n 김동욱\n 감독대행\n 2001.12.27-2002.4.30\n 전창진\n 4대\n 2002.5.1-2009.4.22\n 전창진\n 5대\n 2009.4.23-2013.3.11\n 강동희\n 감독대행\n 2013.3.12-2013.4.29\n 김영만\n 6대\n 2013.4.30-2014.2.1\n 이충희\n 7대\n 2014.2.2-2017.4.14\n 김영만\n 8대\n 2017.4.21-2023.1.5\n 이상범\n 대행\n 2023.1.6 - 2023.4.11\n 김주성\n 9대\n 2023.4.12 -\n 김주성', '남북 전쟁 후 미국은 북부를 중심으로 하는 하나의 큰 국민경제가 정리되었다. 1869년, 오마하와 새크라멘토를 잇는 최초의 대륙횡단철도가 개통되면서

In [77]:
df_long['section_text_clean'].apply(lambda x: len(x)).describe()

count    216720.000000
mean        330.814752
std         987.262379
min           0.000000
25%          42.000000
50%         116.000000
75%         306.000000
max      109620.000000
Name: section_text_clean, dtype: float64

In [68]:
df_long['section_text'].apply(lambda x: len(x)).describe()

count    216720.000000
mean        351.246189
std        1055.865494
min           0.000000
25%          48.000000
50%         124.000000
75%         320.000000
max      112885.000000
Name: section_text, dtype: float64

In [74]:
df_long['section_text_clean'][:10].tolist()

['로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는 순수 아마추어 축구팀이다. 현재 USL 프리미어 디벨로프먼트 리그에 참가하고 있으며 성적은 중위권이다. 창단 당시에는 로스앤젤레스 스톰, 2008년부터 2009년까지는 로스앤젤레스 레전즈였다가 2010년 크루스 아술과 제휴 계약을 맺으면서 이름도 현재의 이름으로 바꾸었다.\n분류:USL 리그 2 구단\n분류:2010년 설립된 축구단\n분류:로스앤젤레스 아줄 레전즈',
 '이케다 쇼헤이 (1981년 4월 27일 ~ )는 일본의 전 축구 선수이다. 과거 시미즈 에스펄스, 산프레체 히로시마, 베갈타 센다이, 제프 유나이티드 지바, 에히메 FC, FC 기후에서 활동하였다.',
 '* \n분류:1981년 출생\n분류:살아있는 사람\n분류:일본의 남자 축구 선수\n분류:일본 남자 청소년 축구 국가대표팀 선수\n분류:J1리그의 축구 선수\n분류:J2리그의 축구 선수\n분류:시미즈 에스펄스의 축구 선수\n분류:산프레체 히로시마의 축구 선수\n분류:베갈타 센다이의 축구 선수\n분류:제프 유나이티드 이치하라 지바의 축구 선수\n분류:에히메 FC의 축구 선수\n분류:FC 기후의 축구 선수\n분류:아시안 게임 축구 메달리스트\n분류:2002년 아시안 게임 축구 참가 선수\n분류:일본의 아시안 게임 은메달리스트\n분류:남자 축구 수비수\n분류:시즈오카시 출신\n분류:시즈오카현 출신 축구 선수',
 '심고(心告)는 진리(眞理) 앞에 자신의 참뜻을 고백하는 불교 용어이다. 그 방법은 "천지(天地)·부모(父母)여 하감(下鑑)하소서, 동포(同胞)·법률(法律)이여 응감(應鑑)하소서. 은혜를 입은 저는 진심으로 진리(眞理) 부처님 앞에 고백하나이다"라고 한 다음 즐거운 일을 당하면 감사를 올리고 괴로운 일을 당하면 사죄를 올리며 결정하기 어려운 일을 당할 때는 결정될 심고를 올려 진리에 의지하고 상의하며 힘을 얻으려는 것을 가리킨다.',
 '분류:불교 용어',
 '섬네일\n빌라 에스쿠데로 플렌테이션(Villa Escu

In [79]:
df_long['section_text_clean_len'] = df_long['section_text_clean'].apply(lambda x: len(x))
df_long['section_text_clean_len'].describe()

count    216720.000000
mean        330.814752
std         987.262379
min           0.000000
25%          42.000000
50%         116.000000
75%         306.000000
max      109620.000000
Name: section_text_clean_len, dtype: float64

In [81]:
df_long['section_text_clean_len'].quantile([0.5, 0.9, 0.95, 0.99, 0.995, 0.999]).to_dict()


{0.5: 116.0,
 0.9: 712.0,
 0.95: 1200.0499999999884,
 0.99: 3490.0,
 0.995: 5244.404999999999,
 0.999: 11964.711000000534}

In [82]:
model_name = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [88]:
def add_tok_len(batch):
    enc = tokenizer(
        batch['section_text_clean'],
        add_special_tokens=True,   # 일관되게 (분포 볼 때 보통 True)
        truncation=False,
        return_attention_mask=False,
    )
    return {"tok_len": [len(ids) for ids in enc["input_ids"]]}


ds_clean = Dataset.from_pandas(df_long[['section_text_clean']].dropna())
ds_len = ds_clean.map(
    add_tok_len,
    batched=True,
    batch_size=256,
    num_proc=8,
    desc="token_len",
)

token_len (num_proc=8):   0%|          | 0/216720 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (10781 > 8192). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (13616 > 8192). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (9363 > 8192). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (24663 > 8192). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (11720 > 8192). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence 

In [101]:
ds_len[0]

{'section_text_clean': '로스앤젤레스 아줄 레전즈는 미국 캘리포니아주 로스앤젤레스를 연고지로 하는 순수 아마추어 축구팀이다. 현재 USL 프리미어 디벨로프먼트 리그에 참가하고 있으며 성적은 중위권이다. 창단 당시에는 로스앤젤레스 스톰, 2008년부터 2009년까지는 로스앤젤레스 레전즈였다가 2010년 크루스 아술과 제휴 계약을 맺으면서 이름도 현재의 이름으로 바꾸었다.\n분류:USL 리그 2 구단\n분류:2010년 설립된 축구단\n분류:로스앤젤레스 아줄 레전즈',
 'tok_len': 143}

In [92]:
tok = np.array(ds_len['tok_len'])
print(pd.Series(tok).describe())
print("quantiles:", {q: int(np.quantile(tok, q)) for q in [0.5, 0.9, 0.95, 0.99, 0.995, 0.999]})

count    216720.000000
mean        184.693785
std         551.015682
min           2.000000
25%          26.000000
50%          67.000000
75%         171.000000
max       67768.000000
dtype: float64
quantiles: {0.5: 67, 0.9: 394, 0.95: 659, 0.99: 1914, 0.995: 2851, 0.999: 6727}


In [93]:
for t in [256, 384, 512, 768, 1024, 2048]:
    print(t, (tok > t).mean())

256 0.16557308970099668
384 0.10288390550018457
512 0.07075489110372832
768 0.04027316352897748
1024 0.026448874123292727
2048 0.008891657438169066


In [95]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

...

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

### 데이터셋

In [113]:
ds = load_dataset("lcw99/wikipedia-korean-20240501", split="train")
df = ds.to_pandas()

df['doc_id'] = df.index

df_clean = df[['doc_id', 'title', 'text']].copy()

df_clean = df_clean[df_clean['text'].str.strip().astype(bool)]

print(df_clean.head())

   doc_id  title                                               text
0       0  지미 카터  '''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ...
1       1     수학  '''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변화 등의 ...
2       2  수학 상수  '''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의 반대...
3       3     문학  파일:Fragonard, The Reader.jpg|섬네일|250px|장오노레 프라...
4       4  나라 목록  스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n이 목록에 ...


In [114]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515425 entries, 0 to 515424
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   doc_id  515425 non-null  int64 
 1   title   515425 non-null  object
 2   text    515425 non-null  object
dtypes: int64(1), object(2)
memory usage: 11.8+ MB


In [117]:
import re
from typing import List, Dict

def chunk_text_tokenwise(
    text: str,
    title: str,
    doc_id: int,
    tokenizer,
    max_tokens: int = 512,
    overlap: int = 50,
    safety_margin: int = 8,
    min_tokens: int = 80,
) -> List[Dict]:
    """
    - chunk 대상: text
    - 최종 입력은 title + chunk_text 를 가정하므로,
      text chunk는 effective_max 토큰으로 제한
    - overlap은 토큰 단위로 보장
    """

    if not isinstance(text, str) or not text.strip():
        return []

    title = "" if title is None else str(title).strip()
    title_tok = tokenizer.encode(title, add_special_tokens=False) if title else []
    effective_max = max(32, max_tokens - len(title_tok) - safety_margin)  # 최소 32 보장

    # 문장/문단 단위 split
    # - \n+ 로 문단을 나누고, 마침표/물음표/느낌표 뒤 공백에서도 나눔
    units = re.split(r'(?<=[.!?])\s+|\n+', text)
    units = [u.strip() for u in units if u and u.strip()]

    chunks = []
    cur_tokens = []
    chunk_idx = 0

    def flush_chunk(tokens):
        nonlocal chunk_idx
        if not tokens:
            return
        chunk_text = tokenizer.decode(tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
        if not chunk_text:
            return
        chunks.append({
            "chunk_id": f"{doc_id}_{chunk_idx}",
            "doc_id": doc_id,
            "title": title,
            "text": chunk_text,
            "offset": chunk_idx,
            "token_len": len(tokens),
        })
        chunk_idx += 1

    for u in units:
        u_tokens = tokenizer.encode(u, add_special_tokens=False)
        if not u_tokens:
            continue

        # 유닛 자체가 effective_max보다 길면: 토큰 슬라이딩으로 쪼갬
        if len(u_tokens) > effective_max:
            # 현재 누적분 먼저 flush
            flush_chunk(cur_tokens)
            cur_tokens = []

            step = max(1, effective_max - overlap)
            for i in range(0, len(u_tokens), step):
                sub = u_tokens[i:i + effective_max]
                flush_chunk(sub)
            continue

        # 누적 시 초과하면 flush + overlap 토큰 유지
        if len(cur_tokens) + len(u_tokens) > effective_max and cur_tokens:
            flush_chunk(cur_tokens)

            # 토큰 단위 overlap 유지
            keep = min(overlap, len(cur_tokens))
            cur_tokens = cur_tokens[-keep:]  # 다음 chunk 시작에 붙일 overlap

        cur_tokens.extend(u_tokens)

    # 마지막 flush
    flush_chunk(cur_tokens)

    # 너무 짧은 chunk 후처리(옵션): min_tokens 미만이면 앞 chunk에 합치기
    if min_tokens and len(chunks) >= 2:
        merged = []
        for c in chunks:
            if merged and c["token_len"] < min_tokens:
                # 이전 chunk에 합치기
                prev = merged[-1]
                prev_text = prev["text"] + " " + c["text"]
                prev_tokens = tokenizer.encode(prev_text, add_special_tokens=False)

                # 다시 effective_max 맞추기(넘치면 그냥 유지하거나, 뒤를 자르는 정책 선택)
                if len(prev_tokens) <= effective_max:
                    prev["text"] = prev_text
                    prev["token_len"] = len(prev_tokens)
                else:
                    # 넘치면: 합치지 않고 그냥 독립 chunk로 둠(보수적)
                    merged.append(c)
            else:
                merged.append(c)
        chunks = merged

    return chunks

In [ ]:
# all_chunk_records = []

# print("청킹 시작")

# for text, title, doc_id in tqdm(zip(df_clean['text'], df_clean['title'], df_clean['doc_id']), total=len(df_clean)):
    
#     # 작성하신 함수 호출
#     chunks = chunk_text_tokenwise(
#         text=text,
#         title=title,
#         doc_id=doc_id,
#         tokenizer=tokenizer,  # 미리 로드해둔 토크나이저 전달
#         max_tokens=512,
#         overlap=50
#     )
    
#     # 결과 리스트에 추가 (chunks는 리스트 형태이므로 extend 사용)
#     all_chunk_records.extend(chunks)

# df_chunks = pd.DataFrame(all_chunk_records)

# print(f"청킹 완료! 총 청크 개수: {len(df_chunks)}")
# print(df_chunks.head())

청킹 시작


  0%|          | 0/515425 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [130]:
df_chunks['token_len'].describe()

count    2325.000000
mean      380.468817
std       135.481123
min        53.000000
25%       269.000000
50%       460.000000
75%       488.000000
max       527.000000
Name: token_len, dtype: float64

In [140]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from joblib import Parallel, delayed, cpu_count
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def process_batch(batch_df):
    batch_results = []
    # 전역 tokenizer 사용
    for text, title, doc_id in zip(batch_df['text'], batch_df['title'], batch_df['doc_id']):
        # 기존 로직 그대로 수행
        chunks = chunk_text_tokenwise(
            text=text, 
            title=title, 
            doc_id=doc_id, 
            tokenizer=tokenizer, 
            max_tokens=512, 
            overlap=50
        )
        batch_results.extend(chunks)
    return batch_results

BATCH_SIZE = 1000  # 한 번에 1000개씩 처리 (적절한 크기)
num_cores = cpu_count()


num_batches = len(df_clean) // BATCH_SIZE + 1
batches = np.array_split(df_clean, num_batches)

print(f"CPU {num_cores}개 사용 / 총 {len(batches)}개 배치로 묶어서 처리 시작...")


results = Parallel(n_jobs=-1, backend='loky')(
    delayed(process_batch)(batch)
    for batch in tqdm(batches)
)

all_chunk_records = [chunk for sublist in results for chunk in sublist]
df_chunks = pd.DataFrame(all_chunk_records)

print(f"청킹 완료! 총 청크 개수: {len(df_chunks)}")
print(df_chunks.head())

CPU 5개 사용 / 총 516개 배치로 묶어서 처리 시작...


/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/516 [00:00<?, ?it/s]

청킹 완료! 총 청크 개수: 1165197
  chunk_id  doc_id  title                                               text  \
0      0_0       0  지미 카터  '''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ...   
1      0_1       0  지미 카터  주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다....   
2      0_2       0  지미 카터  6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 ...   
3      0_3       0  지미 카터  과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 ...   
4      0_4       0  지미 카터  자들 같은 인권유린범죄자를 재판소로 회부하여 국제적인 처벌을 받게 하는 등 인권 신...   

   offset  token_len  
0       0        457  
1       1        485  
2       2        433  
3       3        483  
4       4        476  


In [143]:
df_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165197 entries, 0 to 1165196
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   chunk_id   1165197 non-null  object
 1   doc_id     1165197 non-null  int64 
 2   title      1165197 non-null  object
 3   text       1165197 non-null  object
 4   offset     1165197 non-null  int64 
 5   token_len  1165197 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 53.3+ MB


In [144]:
df_chunks.head()

,chunk_id,doc_id,title,text,offset,token_len
0,0_0,0,지미 카터,"'''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ...",0,457
1,0_1,0,지미 카터,주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다....,1,485
2,0_2,0,지미 카터,"6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 ...",2,433
3,0_3,0,지미 카터,과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 ...,3,483
4,0_4,0,지미 카터,자들 같은 인권유린범죄자를 재판소로 회부하여 국제적인 처벌을 받게 하는 등 인권 신...,4,476


In [ ]:
# def process_single_row(args):
#     text, title, doc_id = args
#     # 기존 함수 호출 (tokenizer는 전역 변수로 있는 것을 사용)
#     return chunk_text_tokenwise(
#         text=text, 
#         title=title, 
#         doc_id=doc_id, 
#         tokenizer=tokenizer, 
#         max_tokens=512, 
#         overlap=50
#     )

# if __name__ == '__main__':

#     os.environ["TOKENIZERS_PARALLELISM"] = "false"

#     data_inputs = list(zip(df_clean['text'], df_clean['title'], df_clean['doc_id']))
    
#     num_cores = cpu_count()
#     print(f" 총 {num_cores}개의 CPU 코어")

#     all_chunk_records = []
    
#     with Pool(processes=num_cores) as pool:
#         # imap: 순서대로 결과를 뱉어줌 (tqdm과 연동하기 좋음)
#         for chunks in tqdm(pool.imap(process_single_row, data_inputs, chunksize=100), total=len(data_inputs)):
#             all_chunk_records.extend(chunks)

#     df_chunks = pd.DataFrame(all_chunk_records)
#     print(f"청킹 완료! 총 청크 개수: {len(df_chunks)}")

In [ ]:
import pandas as pd
import numpy as np
import faiss
from FlagEmbedding import BGEM3FlagModel
from tqdm.auto import tqdm
import pickle

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

batch_size = 32
text_list = df_chunks['text'].tolist()

all_dense_embeddings = []
all_sparse_embeddings = []

print(f"총 {len(text_list)}개 청크 임베딩")
for i in tqdm(range(0, len(text_list), batch_size)):
    batch_texts = text_list[i : i + batch_size]
    
    output = model.encode(
        batch_texts, 
        batch_size=batch_size, 
        max_length=512,
        return_dense=True, 
        return_sparse=True,
        return_colbert_vecs=False
    )
    
    all_dense_embeddings.append(output['dense_vecs'])
    all_sparse_embeddings.extend(output['lexical_weights'])

print("FAISS 인덱싱 중...")
dense_vectors = np.vstack(all_dense_embeddings).astype('float32')
dimension = dense_vectors.shape[1]

# L2 normalize for cosine similarity (IP와 동일 효과)
faiss.normalize_L2(dense_vectors)
index = faiss.IndexFlatIP(dimension)
index.add(dense_vectors)

print(f"FAISS 인덱스 생성 완료 (총 {index.ntotal}개)")

# 4. 저장

faiss.write_index(index, "wikipedia_bge_m3.index")

with open("wikipedia_sparse_vecs.pkl", "wb") as f:
    pickle.dump(all_sparse_embeddings, f)

df_chunks[['text', 'doc_id', 'title', 'chunk_idx']].to_parquet(
    "wikipedia_chunks_meta.parquet",
    index=False
)

print("\n🎉 모든 작업 완료!")
print("📁 생성된 파일:")
print("  1. wikipedia_bge_m3.index (Dense vectors)")
print("  2. wikipedia_sparse_vecs.pkl (Sparse vectors)")
print("  3. wikipedia_chunks_meta.parquet (메타데이터)")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

총 1165197개 청크 임베딩


  0%|          | 0/36413 [00:00<?, ?it/s]




pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 22.94it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.






Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00,  4.17it/s]



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 23.55it/s]






Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.65it/s]



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 28.92it/s]






Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s]



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 29.71it/s]






Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.61it/s]



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 12.57it/s]






Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.61it/s]



pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 28.88it/s]






Inference Embe